<a href="https://colab.research.google.com/github/MeenakshiVeeranki/FMML_MODULE1_LAB1/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1.**Increase Percentage of Validation Set**:

If you allocate a larger portion of your data to the validation set, you have fewer training examples. This can lead to a few outcomes:

**Pros**:

You get a more robust estimate of your model's performance because the validation set is larger and more representative. You might detect overfitting more easily since the model has less data to memorize.

**Cons**:

With fewer training examples, your model may not generalize as well to unseen data, potentially reducing training performance. You might need a larger overall dataset to maintain an adequate number of training samples. Reduce Percentage of Validation Set:

Allocating a smaller portion to the validation set leaves more data for training. This can also have various implications:

**Pros**:

Your model has more data to learn from, which can lead to better training performance. You can work with smaller datasets if needed.

**Cons**:

The estimate of your model's performance on unseen data (validation set) might be less reliable because of the smaller validation set. Overfitting might be less apparent, as the model has more training data to fit the training set closely.

The choice of the percentage for the validation set depends on your specific problem, dataset size, and goals. It often involves a trade-off between having a more reliable estimate of model performance and ensuring that your model has sufficient data for training. Cross-validation techniques can also help mitigate some of these issues by repeatedly splitting the data into training and validation sets.

2.
1**.Larger Training Set:**

If you allocate a larger portion of your data to the training set, your model has more data to learn from. This can help the model capture underlying patterns and relationships in the data more effectively.
With a well-trained model, you can often predict the accuracy on the test set more accurately because the model has seen more diverse examples during training, potentially leading to better generalization.


2.**Larger Validation Set:**

A larger validation set provides a more reliable estimate of your model's performance during training. It ensures that the estimate is based on a broader sample of your data.
This can lead to a more accurate prediction of the model's performance on the test set because your validation set is a better representation of unseen data.

3.**Trade-offs:**

There's often a trade-off between the size of the training and validation sets. If you make the validation set very large, you may reduce the amount of data available for training. Conversely, if you allocate too much data for training, your validation set may be too small to provide a reliable estimate of performance.

Both the size of the training and validation sets can impact your ability to predict the accuracy on the test set. A larger training set can lead to a better-trained model, while a larger validation set can provide a more accurate estimate of performance. Balancing these factors depends on your specific dataset, but it's important to strike a reasonable balance to ensure both effective training and reliable model evaluation. Cross-validation techniques can also be useful in cases where you want to use all your data effectively for both training and validation.

3.The percentage of data to reserve for the validation set depends on various factors, including the size of your dataset and the specific problem you're working on. There isn't a one-size-fits-all answer, but here are some general guidelines to help you balance the two factors:

**Training Set Size:**

A common split ratio is 80% for training and 20% for validation. This is a good starting point for moderate-sized datasets (thousands to tens of thousands of samples).

**Smaller Datasets:**

If you have a relatively small dataset (a few hundred samples), you might consider using a larger portion for validation, such as a 70-30 or even 60-40 split. This provides a more reliable estimate of performance.

**Larger Datasets:**

For very large datasets (hundreds of thousands or more), you can allocate a smaller percentage for validation, like 90-10 or 95-5. The sheer volume of training data can compensate for the smaller validation set.

**Cross-Validation:**

If you want a robust estimate of your model's performance and have enough data, consider using techniques like k-fold cross-validation. This involves splitting your data into k subsets, training and validating k times, and averaging the results.

**Stratified Split:**

In classification tasks with imbalanced classes, consider using stratified sampling to ensure that the validation set maintains the same class distribution as the entire dataset.

**Experimentation:**

Ultimately, the best percentage for your validation set may require experimentation. You can try different splits and evaluate the impact on your model's performance.

Remember that there's often a trade-off between the size of the training and validation sets. You want a sufficiently large training set for effective model learning, but you also want a reliable estimate of performance from your validation set. The key is to find a balance that suits your specific data and task while ensuring that your model neither underfits nor overfits.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Yes, averaging validation accuracy across multiple splits of your dataset can provide more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly referred to as cross-validation.

Cross-validation helps mitigate the potential bias and variance that can occur when using a single train-test split. Here's how it works:

**Data Splitting:**

You divide your dataset into multiple subsets or "folds." A common choice is k-fold cross-validation, where the data is divided into k equal-sized folds.

**Training and Validation**:

You train your model k times, each time using k-1 folds for training and the remaining fold for validation. This process ensures that each data point gets a chance to be in both the training and validation sets.

**Performance Metrics:**

For each fold, you compute the validation accuracy (or other relevant performance metrics).

**Averaging:**

Finally, you calculate the average of the validation accuracies obtained from all the folds. This average is often considered a more robust estimate of your model's performance than using a single validation set.

By averaging across multiple splits, you reduce the likelihood that your model's performance is overly influenced by the specific random choice of the validation set in a single split. It provides a more stable estimate of how well your model is likely to perform on unseen data.

2.The key point is that cross-validation simulates the process of training and testing on multiple subsets of your training data, which provides a more stable and reliable assessment of your model's generalization performance. However, it doesn't eliminate the need for a separate, held-out test set to estimate the model's performance on truly unseen data.

Here's the typical workflow for estimating a model's performance:

**Training and Validation (Cross-Validation)**:

You use cross-validation to assess your model's performance on various subsets of your training data, providing you with a more reliable estimate of how well your model is likely to generalize to similar data.

**Hyperparameter Tuning:**

You can also use cross-validation to tune hyperparameters, ensuring that your model is optimized for the task.

**Test Set Evaluation:**

After you've trained and validated your model using cross-validation, you reserve a separate, held-out test set that your model has never seen during training or validation. You then evaluate your model's performance on this test set to estimate its performance on truly unseen data.

The test set evaluation gives you the most accurate estimate of your model's performance on new, unseen data. It's important to keep this test set separate from your training and validation data to avoid any data leakage and ensure a fair assessment of your model's generalization abilities.

Cross-validation provides a more reliable estimate of a model's performance on your training data, but to estimate its true test accuracy, you need to use a separate test set.

3.The number of iterations (or epochs) in the training process can have an impact on the estimate of your model's performance, but it doesn't necessarily mean that a higher number of iterations will always result in a better estimate. The relationship between the number of iterations and the estimate of your model's performance is more nuanced and depends on several factors:

**Underfitting vs. Overfitting:**

The number of iterations affects how well your model fits the training data. Too few iterations can result in underfitting, where your model do:esn't capture the underlying patterns in the data. Too many iterations can lead to overfitting, where your model fits the noise in the training data and performs poorly on unseen data.

**Early Stopping:**

It's common practice to monitor the model's performance on a validation set during training and stop when the performance starts to degrade. This prevents overfitting and ensures that your model generalizes well. In this case, increasing the number of iterations beyond the point where performance on the validation set starts to degrade won't necessarily improve your estimate.

**Computational Resources:**

Training a model with a higher number of iterations can be computationally expensive and time-consuming. There's often a trade-off between computational cost and the marginal improvement in performance. You'll need to consider practical constraints when deciding on the number of iterations.

**Dataset Size:**

The size of your dataset also plays a role. If you have a large dataset, you might converge to a good solution with fewer iterations. In contrast, a smaller dataset might require more iterations to capture the underlying patterns.

**Learning Rate and Optimization Algorithm:**

The choice of learning rate and optimization algorithm can interact with the number of iterations. For example, if your learning rate is too high, your model might not converge even with a high number of iterations.

You often determine the optimal number of iterations (epochs) through a combination of techniques:

 ▶ Use cross-validation to assess the model's performance with different
     numbers of iterations.
 ▶ Implement early stopping based on the performance on a validation set
    to prevent overfitting.
 ▶ Monitor training progress with learning curves to see if further
    iterations are improving performance or leading to overfitting.

4.Increasing the number of iterations in training may help to some extent when dealing with a very small training dataset or validation dataset, but it is not a guaranteed solution to address the challenges associated with limited data. Here are some considerations:

**Advantages of Increasing Iterations with Small Datasets:**

Overfitting Mitigation: With a small dataset, the risk of overfitting (where the model fits the noise in the data) is higher. Increasing the number of iterations and training for longer can help the model to better fit the available training data, potentially reducing overfitting.

**Fine-Tuning**:

 If you have a pre-trained model (e.g., in transfer learning), increasing the iterations during fine-tuning on a small dataset can help the model adapt more to the specific characteristics of the limited data.

**Limitations and Caveats:**

Data Quality: Simply increasing iterations may not compensate for a lack of data quality. If your small dataset is noisy or unrepresentative, no amount of training iterations can overcome this fundamental issue.

**Risk of Overfitting:**

 While longer training can help mitigate overfitting to some extent, there's a point where the model may start to overfit the limited training data more severely. You must closely monitor the model's performance on a separate validation set and employ techniques like early stopping to prevent overfitting.

**Limited Generalization:**

Even with more iterations, the model's ability to generalize to unseen data may remain limited due to the small dataset's inherent constraints. It might still struggle with capturing the full complexity of the underlying problem.

**Data Augmentation:**

In cases of small datasets, data augmentation techniques can often be more effective than simply increasing the number of iterations. Data augmentation artificially increases the effective size of your training dataset by applying various transformations to your existing data.

**Transfer Learning:**

 If your dataset is very small, consider using transfer learning with a pre-trained model. Transfer learning allows you to leverage knowledge from a larger dataset, which can be highly beneficial even with a limited amount of data.

While increasing the number of iterations can be a helpful strategy when dealing with small datasets, it should be part of a broader set of strategies that may include data augmentation, regularization techniques, and, in particular cases, transfer learning. Careful monitoring of model performance, validation, and an understanding of the data quality are crucial when working with limited data to ensure you are not overfitting or making unreliable assumptions about the underlying data distribution.

